In [1]:
import os, sys

import sgpp, sgml, dproc, sgutil
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print(sys.version)

from sklearn.pipeline import make_pipeline

3.12.9 (main, Mar 15 2025, 13:36:28) [GCC 13.3.0]


In [2]:
X_to_cat = [
    ['Episode_Length_minutes'], ['Guest_Popularity_percentage'], ['Host_Popularity_percentage'], ['Number_of_Ads']
]

In [3]:
sc = sgutil.SGCache('img', 'result', 'model')
p = make_pipeline(
    sgpp.PolarsProcessor(),
    sgpp.ExprProcessor({
        'ELm_num': (pl.col('Episode_Length_minutes').fill_null(pl.col('Episode_Length_minutes').mean()).clip(5, 120) - 5) / 115,
        'GP': (pl.col('Guest_Popularity_percentage').fill_null(pl.col('Guest_Popularity_percentage').clip(0,100).mean()).clip(0, 100) / 100),
        'HP': pl.col('Host_Popularity_percentage').clip(0, 100) / 100,
        'NAd': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0.0) /3 ,
        'Number_of_Ads': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0).cast(pl.Int8),
        'ELm_na': pl.col('Episode_Length_minutes').is_null(),
        'GPp_na': pl.col('Guest_Popularity_percentage').is_null(),
        'ELm_sqrt': ((pl.col('Episode_Length_minutes').fill_null(pl.col('Episode_Length_minutes').mean()).clip(5, 120) - 5) / 115)  ** 0.5
    }),
    sgpp.PandasConverter(index_col = 'id'),
    sgpp.ApplyWrapper(
        sgpp.CatCombiner2(X_to_cat), pd.Series(X_to_cat).explode().tolist()
    )
)
p.fit(['data/train.csv'])
df_train = p.transform(['data/train.csv'])
df_test = p.transform(['data/test.csv'])

In [4]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold, cross_validate, cross_val_score, ShuffleSplit
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector
kf = KFold(5, shuffle = True, random_state = 123)
ss = ShuffleSplit(1, train_size = 0.8, random_state = 123)
target = 'Listening_Time_minutes'
X_all = [i for i in df_train.columns if i != target]

def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index),
    'score_func': lambda df, prds: root_mean_squared_error(df[target], prds),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': target,
}

lr_adapter = sgml.SklearnAdapter(LinearRegression)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor, progress = 50)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor, progress = 50)

In [58]:
df_train_1['resi'].std()

10.683458

In [6]:
from itertools import combinations
X = [
    'Episode_Length_minutes', 'Podcast_Name', 'Episode_Title', 'Genre',
    'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
    'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'    
]

In [22]:
X_1 = [list(i) for i in combinations(X, len(X) - 1)]

In [23]:
df_train.groupby(X_1[0], observed = True)[target].agg(
    ['mean', 'count', 'var']
).sort_values('count', ascending = False).dropna().sort_values('count', ascending = False)

,,,,,,,,,mean,count,var
Episode_Length_minutes,Podcast_Name,Episode_Title,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,,,
"60.89,",Humor Hub,Episode 13,Comedy,"50.89,",Wednesday,Morning,"nan,","1,",46.856369,3,0.0
"64.92,",Crime Chronicles,Episode 16,True Crime,"78.48,",Friday,Evening,"40.54,","2,",43.571674,3,0.0
"44.66,",Tech Talks,Episode 80,Technology,"52.66,",Sunday,Night,"55.77,","0,",30.340000,2,0.0
"nan,",Sports Weekly,Episode 80,Sports,"37.4,",Thursday,Night,"52.84,","1,",91.800003,2,0.0
"67.11,",Sport Spot,Episode 55,Sports,"42.15,",Wednesday,Night,"nan,","1,",59.394821,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...
"117.36,",Digital Digest,Episode 71,Technology,"93.11,",Thursday,Morning,"72.53,","1,",74.223129,2,0.0
"42.07,",Global News,Episode 66,News,"83.82,",Saturday,Evening,"nan,","3,",20.749889,2,0.0
"6.85,",Mystery Matters,Episode 54,True Crime,"22.8,",Monday,Morning,"36.9,","0,",0.128480,2,0.0


In [66]:
d = []
for i in X_1:
    d.append((
        i, df_train.groupby(i, observed = True)[target].agg(
            ['sum', 'count', 'var']
        ).dropna()
    ))

In [76]:
df_train.join(
    s_tmp1.rename(columns = lambda x: (0, x)), on = X_1[1]
).join(
    s_tmp2.rename(columns = lambda x: (1, x)), on = X_1[2]
)

,Podcast_Name,Episode_Title,Genre,Publication_Day,Publication_Time,Episode_Sentiment,Listening_Time_minutes,ELm_num,GP,HP,...,Episode_Length_minutes,Guest_Popularity_percentage,Host_Popularity_percentage,Number_of_Ads,"(0, sum)","(0, count)","(0, var)","(1, sum)","(1, count)","(1, var)"
id,,,,,,,,,,,,,,,,,,,,,
0,Mystery Matters,Episode 98,True Crime,Thursday,Night,Positive,31.419979,0.517432,0.522361,0.7481,...,"nan,","nan,","74.81,","0,",NaN,NaN,NaN,NaN,NaN,NaN
1,Joke Junction,Episode 26,Comedy,Saturday,Afternoon,Negative,88.012413,0.998261,0.759500,0.6695,...,"119.8,","75.95,","66.95,","2,",NaN,NaN,NaN,NaN,NaN,NaN
2,Study Sessions,Episode 16,Education,Tuesday,Evening,Negative,44.925308,0.599130,0.089700,0.6997,...,"73.9,","8.97,","69.97,","0,",NaN,NaN,NaN,NaN,NaN,NaN
3,Digital Digest,Episode 45,Technology,Monday,Morning,Positive,46.278240,0.540609,0.787000,0.5722,...,"67.17,","78.7,","57.22,","2,",NaN,NaN,NaN,NaN,NaN,NaN
4,Mind & Body,Episode 86,Health,Monday,Afternoon,Neutral,75.610313,0.917478,0.586800,0.8007,...,"110.51,","58.68,","80.07,","3,",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,Learning Lab,Episode 25,Education,Saturday,Morning,Negative,56.870579,0.614435,0.522361,0.6936,...,"75.66,","nan,","69.36,","0,",NaN,NaN,NaN,NaN,NaN,NaN
749996,Business Briefs,Episode 21,Business,Saturday,Night,Neutral,45.462421,0.615217,0.522361,0.3521,...,"75.75,","nan,","35.21,","2,",NaN,NaN,NaN,NaN,NaN,NaN
749997,Lifestyle Lounge,Episode 51,Lifestyle,Thursday,Morning,Negative,15.260000,0.225913,0.848900,0.7858,...,"30.98,","84.89,","78.58,","0,",NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df_train.join(
    s_tmp1.rename(columns = lambda x: x + '_1'), on = X_1[1]
).join(
    s_tmp2.rename(columns = lambda x: x + '_2'), on = X_1[2]
).assign(
    count = lambda x: x['count_1'].fillna(0) + x['count_2'].fillna(0),
).query('count > 0').assign(
    var = lambda x: (x['count_1'] - 1).fillna(0) * x['var_1'].fillna(0) + (x['count_2'] - 1).fillna(0) * x['var_2'].fillna(0),
    sum = lambda x: x['sum_1'].fillna(0) + x['sum_2'].fillna(0)
).drop(columns = ['sum_1', 'count_1', 'var_1', 'sum_2', 'count_2', 'var_2'])

,Podcast_Name,Episode_Title,Genre,Publication_Day,Publication_Time,Episode_Sentiment,Listening_Time_minutes,ELm_num,GP,HP,...,ELm_na,GPp_na,ELm_sqrt,Episode_Length_minutes,Guest_Popularity_percentage,Host_Popularity_percentage,Number_of_Ads,count,var,sum
id,,,,,,,,,,,,,,,,,,,,,
107,Daily Digest,Episode 96,News,Friday,Afternoon,Neutral,9.779750,0.280957,0.0562,0.5778,...,False,False,0.530053,"37.31,","5.62,","57.78,","1,",2.0,0.0,19.559500
189,Laugh Line,Episode 9,Comedy,Wednesday,Morning,Positive,93.921432,0.925826,0.6435,0.8726,...,False,False,0.962199,"111.47,","64.35,","87.26,","1,",2.0,0.0,187.842865
213,Comedy Corner,Episode 48,Comedy,Tuesday,Evening,Neutral,60.943081,0.975565,0.5787,0.3917,...,False,False,0.987707,"117.19,","57.87,","39.17,","0,",2.0,0.0,121.886162
274,Fashion Forward,Episode 44,Lifestyle,Sunday,Night,Neutral,31.527269,0.445739,0.1692,0.7426,...,False,False,0.667637,"56.26,","16.92,","74.26,","1,",2.0,0.0,63.054539
299,Music Matters,Episode 90,Music,Saturday,Evening,Positive,113.808250,0.517432,0.1246,0.8104,...,True,False,0.719328,"nan,","12.46,","81.04,","1,",2.0,0.0,227.616501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749457,Daily Digest,Episode 14,News,Tuesday,Night,Neutral,37.143108,0.491478,0.7029,0.4075,...,False,False,0.701055,"61.52,","70.29,","40.75,","2,",2.0,0.0,74.286217
749543,Lifestyle Lounge,Episode 62,Lifestyle,Wednesday,Evening,Neutral,17.447941,0.230087,0.5942,0.5888,...,False,False,0.479674,"31.46,","59.42,","58.88,","2,",3.0,0.0,52.343822
749840,Global News,Episode 9,News,Friday,Morning,Negative,29.304951,0.298261,0.5754,0.9065,...,False,False,0.546133,"39.3,","57.54,","90.65,","1,",2.0,0.0,58.609901


In [36]:
df_train[X_1[2]].apply(tuple, axis = 1).to_frame().join(s_tmp)

ValueError: cannot join with no overlapping index names

In [32]:
s_tmp

,,,,,,,,,mean,count,var
Episode_Length_minutes,Podcast_Name,Episode_Title,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Number_of_Ads,Episode_Sentiment,,,
"97.54,",Tech Talks,Episode 49,Technology,"27.8,",Tuesday,Night,"1,",Neutral,53.063530,7,0.0
"42.45,",Gadget Geek,Episode 1,Technology,"72.72,",Sunday,Evening,"0,",Negative,35.599560,6,0.0
"59.87,",Educational Nuggets,Episode 35,Education,"66.15,",Thursday,Night,"1,",Negative,46.895329,6,0.0
"34.44,",Current Affairs,Episode 87,News,"48.94,",Thursday,Afternoon,"1,",Negative,27.718880,6,0.0
"81.79,",Detective Diaries,Episode 79,True Crime,"84.63,",Saturday,Morning,"1,",Neutral,65.179871,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...
"110.18,",Finance Focus,Episode 90,Business,"86.03,",Friday,Morning,"1,",Neutral,77.949997,2,0.0
"99.45,",Health Hour,Episode 61,Health,"88.38,",Friday,Night,"1,",Negative,79.655472,2,0.0
"14.61,",Lifestyle Lounge,Episode 54,Lifestyle,"28.11,",Sunday,Night,"0,",Negative,13.489000,2,0.0


In [107]:
df_train_1.groupby([
    'Podcast_Name', 'Episode_Title', 'Genre',
    'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
    'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'    
], observed = True)['resi'].agg(['mean', 'count', 'var']).dropna().sort_values('count', ascending = False)

mean  \
Podcast_Name  Episode_Title Genre  Host_Popularity_percentage Publication_Day Publication_Time Guest_Popularity_percentage Number_of_Ads Episode_Sentiment              
Global News   Episode 96    News   45.509998                  Thursday        Evening          6.480000                    1             Positive            1.239313   
Joke Junction Episode 20    Comedy 60.040001                  Tuesday         Afternoon        78.730003                   0             Negative            5.494633   
Laugh Line    Episode 6     Comedy 85.580002                  Sunday          Evening          97.459999                   1             Neutral             4.743495   
Sport Spot    Episode 44    Sports 29.650000                  Wednesday       Evening          22.540001                   3             Negative           -1.887944   
Funny Folks   Episode 75    Comedy 89.849998                  Sunday          Morning          38.520000                   0             Positive            2.221181   
...                                                                                                                                                               ...   
              Episode 81    Comedy 40.549999                  Wednesday       Night            10.670000                   1             Positive            0.702034   
                                   39.840000                  Thursday        Afternoon        51.459999                   1             Positive            2.309190   
                                   34.180000                  Tuesday         Morning          29.610001                   0             Positive          -21.565681   
              Episode 47    Comedy 29.770000                  Sunday          Night            60.770000                   2             Positive           18.350576   
Tune Time     Episode 70    Music  38.080002                  Sunday          Afternoon        70.889999                   1             Negative            5.663251   

                                                                                                                                                            count  \
Podcast_Name  Episode_Title Genre  Host_Popularity_percentage Publication_Day Publication_Time Guest_Popularity_percentage Number_of_Ads Episode_Sentiment          
Global News   Episode 96    News   45.509998                  Thursday        Evening          6.480000                    1             Positive               7   
Joke Junction Episode 20    Comedy 60.040001                  Tuesday         Afternoon        78.730003                   0             Negative               6   
Laugh Line    Episode 6     Comedy 85.580002                  Sunday          Evening          97.459999                   1             Neutral                6   
Sport Spot    Episode 44    Sports 29.650000                  Wednesday       Evening          22.540001                   3             Negative               5   
Funny Folks   Episode 75    Comedy 89.849998                  Sunday          Morning          38.520000                   0             Positive               5   
...                                                                                                                                                           ...   
              Episode 81    Comedy 40.549999                  Wednesday       Night            10.670000                   1             Positive               2   
                                   39.840000                  Thursday        Afternoon        51.459999                   1             Positive               2   
                                   34.180000                  Tuesday         Morning          29.610001                   0             Positive               2   
              Episode 47    Comedy 29.770000                  Sunday          Night            60.770000                   2             Positive   

In [96]:
df_train_1.groupby([
    'Episode_Length_minutes', 'Podcast_Name', 'Episode_Title', 'Genre',
    'Publication_Day', 'Publication_Time',
    'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'    
], observed = True)['resi'].agg(['std', 'size', 'mean']).dropna().sort_values('size', ascending = False)

,,,,,,,,,std,size,mean
Episode_Length_minutes,Podcast_Name,Episode_Title,Genre,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,,,
45.639999,Health Hour,Episode 99,Health,Sunday,Evening,5.980000,1,Neutral,0.946115,5,4.944539
98.050003,Sport Spot,Episode 51,Sports,Sunday,Night,49.470001,1,Negative,0.349739,4,20.566601
49.139999,Business Briefs,Episode 56,Business,Sunday,Evening,66.650002,1,Positive,0.119714,4,5.920768
92.440002,World Watch,Episode 64,News,Wednesday,Afternoon,71.400002,0,Positive,0.136361,4,-3.584320
79.669998,Learning Lab,Episode 63,Education,Friday,Afternoon,86.269997,1,Neutral,0.380617,4,-2.479351
...,...,...,...,...,...,...,...,...,...,...,...
45.689999,Laugh Line,Episode 27,Comedy,Friday,Morning,54.689999,0,Negative,1.025964,2,1.517778
45.680000,Money Matters,Episode 84,Business,Wednesday,Morning,54.610001,0,Positive,0.007191,2,10.224369
45.660000,Fashion Forward,Episode 88,Lifestyle,Wednesday,Afternoon,65.010002,1,Positive,0.066324,2,12.203983
